Imports

In [85]:
import numpy as np
import random
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Import sentence-transformers for easy BERT embeddings
from sentence_transformers import SentenceTransformer
import os

In [86]:
# Create a directory for embeddings if it doesn't exist
embeddings_dir = "bert_embeddings"
os.makedirs(embeddings_dir, exist_ok=True)

# Filepaths for saved embeddings
train_emb_path = os.path.join(embeddings_dir, "train_embeddings.npy")
val_emb_path = os.path.join(embeddings_dir, "val_embeddings.npy")
test_emb_path = os.path.join(embeddings_dir, "test_embeddings.npy")
scaler_path = os.path.join(embeddings_dir, "scaler.pkl")

Read CSV

In [87]:
# Set seed
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

# Load the dataset
dataset = pd.read_csv("../../datasets/final_dataset.csv")

# First split: train and test
train_texts, test_texts, train_labels, test_labels = train_test_split(
    dataset["Text"], dataset["Label"], test_size=0.2, random_state=42, stratify=dataset["Label"]
)

# Second split: train and validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

In [88]:
# Convert labels to numpy arrays
y_train = np.array(train_labels)
y_val = np.array(val_labels)
y_test = np.array(test_labels)

Process Data

In [89]:
# Load a pre-trained BERT-based sentence transformer model
bert_model = SentenceTransformer('all-MiniLM-L6-v2')  # Smaller BERT model variant

# Check if embeddings already exist
if (os.path.exists(train_emb_path) and 
    os.path.exists(val_emb_path) and 
    os.path.exists(test_emb_path) and
    os.path.exists(scaler_path)):
    
    print("Loading pre-computed BERT embeddings...")
    X_train_bert = np.load(train_emb_path)
    X_val_bert = np.load(val_emb_path)
    X_test_bert = np.load(test_emb_path)
    
    # Load the scaler
    import pickle
    with open(scaler_path, 'rb') as f:
        scaler = pickle.load(f)
    
    print(f"Loaded embeddings: Train shape {X_train_bert.shape}, Val shape {X_val_bert.shape}, Test shape {X_test_bert.shape}")
    
else:
    print("Computing BERT embeddings (this may take a while)...")

    print("Extracting BERT embeddings for training set...")
    X_train_bert = bert_model.encode(train_texts.tolist(), 
                                    show_progress_bar=True, 
                                    batch_size=32)

    print("Extracting BERT embeddings for validation set...")
    X_val_bert = bert_model.encode(val_texts.tolist(), 
                                  show_progress_bar=True, 
                                  batch_size=32)

    print("Extracting BERT embeddings for test set...")
    X_test_bert = bert_model.encode(test_texts.tolist(), 
                                   show_progress_bar=True, 
                                   batch_size=32)
    
    # Save the embeddings
    print("Saving embeddings to disk for future use...")
    np.save(train_emb_path, X_train_bert)
    np.save(val_emb_path, X_val_bert)
    np.save(test_emb_path, X_test_bert)
    
    # Create and save the scaler
    scaler = StandardScaler()
    scaler.fit(X_train_bert)
    
    import pickle
    with open(scaler_path, 'wb') as f:
        pickle.dump(scaler, f)
    
    print("Embeddings and scaler saved successfully!")

Loading pre-computed BERT embeddings...
Loaded embeddings: Train shape (2593, 384), Val shape (649, 384), Test shape (811, 384)


In [90]:
# Normalize the embeddings (optional but recommended)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_bert)
X_val_scaled = scaler.transform(X_val_bert)
X_test_scaled = scaler.transform(X_test_bert)

# Reshape embeddings for LSTM: (samples, time steps, features)
# We'll reshape our embeddings into a sequence format for LSTM
embedding_dim = X_train_scaled.shape[1]  # Should be 384 for this model
time_steps = 4  # Split embedding into 4 parts to create a sequence
features = embedding_dim // time_steps

# Reshape 2D embeddings to 3D for LSTM: (samples, time_steps, features)
X_train = X_train_scaled.reshape(-1, time_steps, features)
X_val = X_val_scaled.reshape(-1, time_steps, features)
X_test = X_test_scaled.reshape(-1, time_steps, features)


In [91]:

# Build model with your original LSTM architecture
model = Sequential([
    # No embedding layer needed since we already have BERT embeddings
    
    LSTM(16, activation="tanh", return_sequences=True, 
         kernel_regularizer=l2(0.003), recurrent_dropout=0.5,
         input_shape=(time_steps, features)),  # Specify input shape
    Dropout(0.4),

    LSTM(8, kernel_regularizer=l2(0.003), recurrent_dropout=0.5, return_sequences=False),
    Dropout(0.3),

    BatchNormalization(),

    Dense(16, activation="relu", kernel_regularizer=l2(0.003)),
    Dropout(0.4),

    Dense(1, activation="sigmoid")
])

learning_rate = 0.0005
epochs = 20
batch_size = 32

# Compile with appropriate learning rate
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

# Add callbacks for training
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.3,
    patience=1,
    min_lr=0.00005
)


c:\Users\Utilizador\miniconda3\envs\daa\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [92]:
# Train the model
print("Training model...")
history = model.fit(
    X_train, y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr]
)


Training model...
Epoch 1/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 7s 15ms/step - accuracy: 0.5069 - loss: 1.0848 - val_accuracy: 0.5193 - val_loss: 0.9858 - learning_rate: 5.0000e-04
Epoch 2/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.4846 - loss: 1.0372 - val_accuracy: 0.5270 - val_loss: 0.9611 - learning_rate: 5.0000e-04
Epoch 3/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5020 - loss: 0.9980 - val_accuracy: 0.5362 - val_loss: 0.9366 - learning_rate: 5.0000e-04
Epoch 4/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5087 - loss: 0.9658 - val_accuracy: 0.5562 - val_loss: 0.9129 - learning_rate: 5.0000e-04
Epoch 5/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5394 - loss: 0.9251 - val_accuracy: 0.5778 - val_loss: 0.8901 - learning_rate: 5.0000e-04
Epoch 6/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5373 - loss: 0.9046 - val_accuracy: 0.6148 - val_loss: 0.8656 - learning_rate: 5.0000e-04
Epoch 7/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accur

In [93]:

# Evaluate on test set
print("Evaluating model...")
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc:.4f}\n")

val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation accuracy: {val_acc:.4f}")

Evaluating model...
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8116 - loss: 0.5139
Test accuracy: 0.8187

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8352 - loss: 0.4801
Validation accuracy: 0.8336


In [94]:


# Function to process new data and make predictions
def predict_on_new_data(new_texts):
    # Extract BERT embeddings
    new_embeddings = bert_model.encode(new_texts, show_progress_bar=True, batch_size=32)
    # Scale embeddings
    new_embeddings_scaled = scaler.transform(new_embeddings)
    # Reshape for LSTM
    new_embeddings_reshaped = new_embeddings_scaled.reshape(-1, time_steps, features)
    # Predict
    return model.predict(new_embeddings_reshaped).flatten()

# Benchmarking
print("Loading benchmark data...")
new_data = pd.read_csv("../../datasets/dataset1_inputs.csv", delimiter="\t")

# Make predictions
print("Making predictions on benchmark data...")
predictions = predict_on_new_data(new_data["Text"].tolist())

# Convert predictions to labels
labels = ["AI" if pred > 0.5 else "Human" for pred in predictions]

# Create output DataFrame
output_df = pd.DataFrame({"ID": new_data["ID"], "Label": labels, "Prediction": predictions})

# Load the correct labels
ground_truth = pd.read_csv("../../datasets/dataset1_outputs.csv", delimiter="\t")

# Merge predictions with ground truth
comparison_df = output_df.merge(ground_truth, on="ID", suffixes=("_predicted", "_actual"))

# Calculate accuracy
accuracy = (comparison_df["Label_predicted"] == comparison_df["Label_actual"]).mean()

# Print results
print(f"Benchmark accuracy: {accuracy:.4f}")

# Show misclassified samples
misclassified = comparison_df[comparison_df["Label_predicted"] != comparison_df["Label_actual"]]
print("\nNumber of misclassified samples:", len(misclassified))


print(comparison_df)

print("Complete!")

Loading benchmark data...
Making predictions on benchmark data...


Batches: 100%|██████████| 1/1 [00:02<00:00,  2.37s/it]


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 871ms/step
Benchmark accuracy: 0.6333

Number of misclassified samples: 11
       ID Label_predicted  Prediction Label_actual
0    D1-1           Human    0.254965        Human
1    D1-2              AI    0.631633           AI
2    D1-3           Human    0.241493        Human
3    D1-4           Human    0.142752           AI
4    D1-5              AI    0.838111        Human
5    D1-6              AI    0.967630           AI
6    D1-7           Human    0.299215        Human
7    D1-8           Human    0.433918           AI
8    D1-9              AI    0.717566        Human
9   D1-10              AI    0.850823           AI
10  D1-11              AI    0.979280        Human
11  D1-12              AI    0.965557           AI
12  D1-13           Human    0.120815        Human
13  D1-14              AI    0.563560           AI
14  D1-15              AI    0.935525        Human
15  D1-16              AI    0.878905           AI
16  D1-17              AI    0